In [304]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('glass.csv')
# convert the data into numpy array
data = data.values 

### Mahanalobis Distance Computation

In [305]:
def mahanalobis_distance(X, mean, inverse_covariance):
    X = X - mean
    result = np.matmul(X, inverse_covariance)
    result = np.matmul(result, X.T)
    result = np.sqrt(result)
    return result

In [306]:
no_of_data_points = data.shape[0]
# Compute the mean and covariance matrix
mean = np.mean(data, axis=0)
covariance_matrix = np.cov(data, rowvar=False)
inverse_covariance_matrix = np.linalg.inv(covariance_matrix)

In [307]:
# Compute the Mahalanobis distance for each data point
mahanalobis_distances = np.zeros(no_of_data_points)
for i in range(no_of_data_points):
    mahanalobis_distances[i] = mahanalobis_distance(data[i], mean, inverse_covariance_matrix)

In [308]:
# Sorting the Mahalanobis distances in ascending order
mahanalobis_distances = np.sort(mahanalobis_distances)
# Threshold array stores the threshold value for each index
threshold = np.zeros(len(mahanalobis_distances))

### Otsu Thresholding for Mahanalobis Distance

In [309]:
# Finding the Otsu threshold value for each index
for i in range(1,len(mahanalobis_distances)):
    weights = np.zeros(2)
    mean = np.zeros(2)
    # finding the weights of the two classes
    weights[0] = i / len(mahanalobis_distances)
    weights[1] = 1 - weights[0]
    # we calculate the mean of the two classes and then find the threshold
    for j in range(len(mahanalobis_distances)):
        if (j<i):
            mean[0] += mahanalobis_distances[j]
        else:
            mean[1] += mahanalobis_distances[j]
    mean[0] /= i
    mean[1] /= (len(mahanalobis_distances) - i)
    # Finding the difference between the two means
    mean_difference = mean[0] - mean[1]
    weights_product = weights[0] * weights[1]
    # Variance between the two classes
    variance = weights_product * mean_difference * mean_difference
    # Finding the threshold value
    threshold[i] = variance


In [310]:
# Finding the index of the maximum threshold value
max_threshold_index = np.argmax(threshold)
# The Otsu threshold value
otsu_threshold = mahanalobis_distances[max_threshold_index]

In [311]:
# finding the no of outliers
no_of_outliers = 0
for data_point in mahanalobis_distances:
    if data_point > otsu_threshold:
        no_of_outliers += 1
print('Otsu threshold value: ', otsu_threshold)
print('No of outliers: ', no_of_outliers)
print('No of inliers: ', len(mahanalobis_distances) - no_of_outliers)
print('Intra class variance: ', threshold[max_threshold_index])
print('Inter class variance: ', threshold[0])
print('Percentage of outliers: ', (no_of_outliers / len(mahanalobis_distances)) * 100 , '%')
print('Percentage of inliers: ', 100 - ((no_of_outliers / len(mahanalobis_distances)) * 100) , '%')

Otsu threshold value:  4.14316806819999
No of outliers:  29
No of inliers:  185
Intra class variance:  1.531628003617358
Inter class variance:  0.0
Percentage of outliers:  13.551401869158877 %
Percentage of inliers:  86.44859813084112 %


### Local Outlier Factor 

In [312]:
# finding the k nearest neighbors for each data point in the data set
def k_nearest_neighbors(X, k):
    no_of_samples = X.shape[0]
    # distance between each pair of data points is stored
    distances = np.zeros((no_of_samples, no_of_samples))
    # finding the distance between each pair of points
    for i in range(no_of_samples):
        for j in range(no_of_samples):
            distances[i, j] = np.linalg.norm(X[i] - X[j])
    knn_indices = np.zeros((no_of_samples, k), dtype=int)
    # finding the k nearest neighbors for each data point
    for i in range(no_of_samples):
        # argsort returns the indices of the sorted array
        # 0th index is skipped because it will be the same as the data point
        knn_indices[i] = np.argsort(distances[i])[1:k+1]
    return knn_indices

In [313]:
# computing the reach distance for each data point to its kth nearest neighbor
def reach_distance(X, knn_indices):
    no_of_samples = X.shape[0]
    reach_distances = np.zeros((no_of_samples, no_of_samples))
    # finds the reach distance for each data point
    for i in range(no_of_samples):
        for j in knn_indices[i]:
            for k in knn_indices[i]:
                # if the data point is not the same as the kth nearest neighbor, then the reach distance is computed
                # else the reach distance is set to the distance between the data point and its kth nearest neighbor
                if k != j:
                    reach_distances[i, j] = max(np.linalg.norm(X[i] - X[j]), reach_distances[i, k])
    return reach_distances

In [314]:
# Define a function to compute the local reachability density of each sample
def lrd(X, knn_indices, reachability_distances, k):
    no_of_samples = X.shape[0]
    lrd = np.zeros(no_of_samples)
    # lrd is computed for each data point
    for i in range(no_of_samples):
        # the mean of the reachability distances of the k nearest neighbors is computed
        for j in range(no_of_samples):
            # 1e-10 is added to avoid division by zero 
            # if the mean is zero, then it will cause division by zero, so adding 1e-10 to avoid the error
            lrd[i] = 1 / (np.mean(reachability_distances[i, knn_indices[i]]) + 1e-10)
    return lrd

In [315]:
# Define a function to compute the local outlier factor of each sample
def lof(X, knn_indices, lrd):
    no_of_samples = X.shape[0]
    lof = np.zeros(no_of_samples)
    # lof is computed for each data point
    for i in range(no_of_samples):
        # the ratio of the lrd of the data point to the lrd of its k nearest neighbors is computed
        lrd_ratios = np.zeros(k)
        for j in range(k):
            # for each k nearest neighbor, we calculate the ratio of the lrd of the data point to the lrd of the k nearest neighbor
            lrd_ratios[j] = lrd[knn_indices[i][j]] / lrd[i]
        # mean of the lrd ratios is the lof value
        lof[i] = np.mean(lrd_ratios)
    return lof

In [316]:
# k is the number of nearest neighbors
k = 5
k_nearest_neighours = k_nearest_neighbors(data, k)
reach_distance = reach_distance(data, k_nearest_neighours)
lrd = lrd(data, k_nearest_neighours, reach_distance, k)
lof = lof(data, k_nearest_neighours, lrd)

### Otsu Thresholding for Local Outlier Factor

In [317]:
# Computing the Otsu threshold
lof = np.sort(lof)
threshold = np.zeros(len(lof))

In [318]:
for i in range(1,len(lof)):
    weights = np.zeros(2)
    mean = np.zeros(2)
    # Finding the weights and mean of each class
    weights[0] = i / len(lof)
    weights[1] = 1 - weights[0]
    for j in range(len(lof)):
        if (j < i):
            mean[0] += lof[j]
        else:
            mean[1] += lof[j]
    mean[0] /= i
    mean[1] /= (len(lof) - i)
    mean_difference = mean[0] - mean[1]
    # Finding the variance between the two classes
    variance = weights[0] * weights[1] * mean_difference * mean_difference
    threshold[i] = variance

In [319]:
# Finding the index of the maximum variance
max_threshold_index = np.argmax(threshold)
# Otsu threshold using the index of the maximum variance
otsu_threshold = lof[max_threshold_index]
# finding the no of outliers

In [320]:
no_of_outliers = 0
for data_point in lof:
    if data_point > otsu_threshold:
        no_of_outliers += 1

In [321]:
print('Otsu threshold value: ', otsu_threshold)
print('No of outliers: ', no_of_outliers)
print('No of inliers:' , len(lof) - no_of_outliers)
print('Intra class variance: ', threshold[max_threshold_index])
print('Inter class variance: ', threshold[0])
print('Percentage of outliers: ', (no_of_outliers / len(lof)) * 100 , '%')
print('Percentage of inliers: ', 100 - ((no_of_outliers / len(lof)) * 100) , '%')


Otsu threshold value:  5.079809867042725
No of outliers:  3
No of inliers: 211
Intra class variance:  0.5551403762366578
Inter class variance:  0.0
Percentage of outliers:  1.4018691588785046 %
Percentage of inliers:  98.5981308411215 %
